# Quick start to run Zephyr 7B Alpha on Google Colab

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
%pip install git+https://github.com/huggingface/transformers.git
%pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-vtr55wkn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-vtr55wkn
  Resolved https://github.com/huggingface/transformers.git to commit 691fd8fdded34497451bc638716eb8fbd484883a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7884053 sha256=d77d1b0e0f204573437274eda2d1362ac4c4b537389cd854a24ee00e2a447253
  Stored in directory: /tmp/pip-ephem-whee

## Load dependencies

In [ ]:
import torch
from transformers import pipeline
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import re


## Download model and load pipeline

In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-alpha", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

## Prepare inputs

In [ ]:

def clean_transcript(transcript):
    # Define regular expression patterns for timestamps and noise
    timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
    noise_pattern = r'Speaker \d+:|\[.*?\]|\n'

    # Remove timestamps and noise
    cleaned_transcript = re.sub(timestamp_pattern, '', transcript)
    cleaned_transcript = re.sub(noise_pattern, '', cleaned_transcript)

    # Split the cleaned transcript into lines
    lines = cleaned_transcript.strip().split('\n')



    return lines[0]

#

In [ ]:
def read_transcript_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        transcript = file.read()
    return transcript


# Provide the path to your transcript file
file_path = '/content/RIOT_-_1_Ring_Rd_BO-003.srt'

transcript = read_transcript_from_file(file_path)


In [ ]:

cleaned_transcript = clean_transcript(transcript)

In [ ]:
print(cleaned_transcript)

﻿1  Cameras.2 All right, we've got my squad over here.3 We're just going to parachute them down the street.4 So kind of form up in a line.5 Jay, I'm on the other side.6 Tech man.7 .8 I'm attacked.9 I'm attacked.10 I'm attacked.11 .12 You've got to put your job, man.13 It's not you.14 .15 Squadron, exit area.16 .17 .18 What do you got?19 .20   .21 Hey, guys, keep it tight.22 Keep it tight.23 .24 .25 How many more people?26    .27 Guys, turn and face that way.28 .29  >> .30 >> Where's McGrail?31 >> I think he's down on his head.32 >> Are you in his squad?33 >> Huh?34 >> Are you in his squad?35 >> I'm in Hogan's squad.36 >> Line together, put the line together.37 >> Where's McGrail?38 >> .39 >> .40 >> All right, what's that Brian?41 >> .42 >> There we go, there we go.43 Nobody through.44 >> .45 >> .46 >> I'm working on a cap, and I'll advise the LECC.47 we need help down there.48 They're throwing bricks now at the opposite.49 >> .50 >> .51 >> I'll check on it.52 >> .53 >> Several vehicles

In [ ]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant". For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are a AI systems that detects confusion in audio transcript.",
    },
    {"role": "user", "content": cleaned_transcript[:1000]},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a AI systems that detects confusion in audio transcript.</s>
<|user|>
﻿1  Cameras.2 All right, we've got my squad over here.3 We're just going to parachute them down the street.4 So kind of form up in a line.5 Jay, I'm on the other side.6 Tech man.7 .8 I'm attacked.9 I'm attacked.10 I'm attacked.11 .12 You've got to put your job, man.13 It's not you.14 .15 Squadron, exit area.16 .17 .18 What do you got?19 .20   .21 Hey, guys, keep it tight.22 Keep it tight.23 .24 .25 How many more people?26    .27 Guys, turn and face that way.28 .29  >> .30 >> Where's McGrail?31 >> I think he's down on his head.32 >> Are you in his squad?33 >> Huh?34 >> Are you in his squad?35 >> I'm in Hogan's squad.36 >> Line together, put the line together.37 >> Where's McGrail?38 >> .39 >> .40 >> All right, what's that Brian?41 >> .42 >> There we go, there we go.43 Nobody through.44 >> .45 >> .46 >> I'm working on a cap, and I'll advise the LECC.47 we need help down there.48 They're throwing bric

In [ ]:
# Each message can have 1 of 3 roles: "system" (to provide initial instructions), "user", or "assistant". For inference, make sure "user" is the role in the final message.
messages = [
    {
        "role": "system",
        "content": "You are an system designed to identify planning issues, aggression, and confusion in audio transcripts. For each identified instance, provide the sentence and a brief explanation."

    },
    {"role": "user", "content": cleaned_transcript[:1000]},
]
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are an system designed to identify planning issues, aggression, and confusion in audio transcripts. For each identified instance, provide the sentence and a brief explanation.</s>
<|user|>
﻿1  Cameras.2 All right, we've got my squad over here.3 We're just going to parachute them down the street.4 So kind of form up in a line.5 Jay, I'm on the other side.6 Tech man.7 .8 I'm attacked.9 I'm attacked.10 I'm attacked.11 .12 You've got to put your job, man.13 It's not you.14 .15 Squadron, exit area.16 .17 .18 What do you got?19 .20   .21 Hey, guys, keep it tight.22 Keep it tight.23 .24 .25 How many more people?26    .27 Guys, turn and face that way.28 .29  >> .30 >> Where's McGrail?31 >> I think he's down on his head.32 >> Are you in his squad?33 >> Huh?34 >> Are you in his squad?35 >> I'm in Hogan's squad.36 >> Line together, put the line together.37 >> Where's McGrail?38 >> .39 >> .40 >> All right, what's that Brian?41 >> .42 >> There we go, there we go.43 Nobody through.44 

## Generate!

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are an system designed to identify planning issues, aggression, and confusion in audio transcripts. For each identified instance, provide the sentence and a brief explanation.</s>
<|user|>
﻿1  Cameras.2 All right, we've got my squad over here.3 We're just going to parachute them down the street.4 So kind of form up in a line.5 Jay, I'm on the other side.6 Tech man.7 .8 I'm attacked.9 I'm attacked.10 I'm attacked.11 .12 You've got to put your job, man.13 It's not you.14 .15 Squadron, exit area.16 .17 .18 What do you got?19 .20   .21 Hey, guys, keep it tight.22 Keep it tight.23 .24 .25 How many more people?26    .27 Guys, turn and face that way.28 .29  >> .30 >> Where's McGrail?31 >> I think he's down on his head.32 >> Are you in his squad?33 >> Huh?34 >> Are you in his squad?35 >> I'm in Hogan's squad.36 >> Line together, put the line together.37 >> Where's McGrail?38 >> .39 >> .40 >> All right, what's that Brian?41 >> .42 >> There we go, there we go.43 Nobody through.44 

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|system|>
You are a AI systems that detects confusion in audio transcript.</s>
<|user|>
﻿1  Cameras.2 All right, we've got my squad over here.3 We're just going to parachute them down the street.4 So kind of form up in a line.5 Jay, I'm on the other side.6 Tech man.7 .8 I'm attacked.9 I'm attacked.10 I'm attacked.11 .12 You've got to put your job, man.13 It's not you.14 .15 Squadron, exit area.16 .17 .18 What do you got?19 .20   .21 Hey, guys, keep it tight.22 Keep it tight.23 .24 .25 How many more people?26    .27 Guys, turn and face that way.28 .29  >> .30 >> Where's McGrail?31 >> I think he's down on his head.32 >> Are you in his squad?33 >> Huh?34 >> Are you in his squad?35 >> I'm in Hogan's squad.36 >> Line together, put the line together.37 >> Where's McGrail?38 >> .39 >> .40 >> All right, what's that Brian?41 >> .42 >> There we go, there we go.43 Nobody through.44 >> .45 >> .46 >> I'm working on a cap, and I'll advise the LECC.47 we need help down there.48 They're throwing bric

In [ ]:
output = outputs[0]["generated_text"]

In [ ]:
input_video_path = "input_video.mp4"

In [ ]:
quoted_strings = re.findall(r'"([^"]*)"', output)

# Search for quoted strings in the transcript and print corresponding timestamp and explanation
for string in quoted_strings:
    pattern = re.compile(rf'\b{re.escape(string)}\b')
    match = re.search(pattern, transcript)
    if match:
        start = max(match.start() - 50, 0)
        end = min(match.end() + 50, len(transcript))
        context = transcript[start:end].strip()
        print(f'String: "{string}"')
        matches = re.findall(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\nSpeaker 0: (.+)', context)

        # Iterate through matches and print timestamps and strings
        for match in matches:
            start_time, end_time, text = match
            print(f"Timestamp: {start_time} --> {end_time}")
            print(f"String: {text}\n")
            output_video_path = f"{text}.mp4"

            start_ms = (int(start_time[:2]) * 3600 + int(start_time[3:5]) * 60 + float(start_time[6:])) * 1000
            end_ms = (int(end_time[:2]) * 3600 + int(end_time[3:5]) * 60 + float(end_time[6:])) * 1000

            ffmpeg_extract_subclip(input_video_path, start_ms, end_ms, targetname=output_video_path)



String: "I'm attacked"
Timestamp: 00:01:50,191 --> 00:01:50,726
String: I'm attacked.

Timestamp: 00:01:51,695 --> 00:01:52,065
String: I'm

String: "Line together, put the line together"
Timestamp: 00:11:04,150 --> 00:11:07,631
String: >> Line together, put the line together.

Timestamp: 00:11:07,631 --> 00:11:25,159
String: >>

String: "Several vehicles"
Timestamp: 00:16:20,969 --> 00:16:27,658
String: >> Several vehicles damaged there.



In [ ]:
end_time

'00:16:27,658'

In [ ]:




# Define the start and end times in seconds
start_time = 10  # Start time in seconds
end_time = 20    # End time in seconds

# Clip the video
ffmpeg_extract_subclip(input_video_path, start_time, end_time, targetname=output_video_path)


'.\n\n53\n00:16:20,969 --> 00:16:27,658\nSpeaker 0: >> Several vehicles damaged there.\n\n54\n00:16:27,658 --> 00:16:33,182'

In [ ]:
matches = re.findall(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\nSpeaker 0: (.+)', input_string)

# Iterate through matches and print timestamps and strings
for match in matches:
    start_time, end_time, text = match
    print(f"Timestamp: {start_time} --> {end_time}")
    print(f"String: {text}\n")


NameError: ignored